In [1]:
from MovieLens import MovieLens
from surprise import KNNBasic
import heapq
from collections import defaultdict
from operator import itemgetter

print('Libraries imported!')

Libraries imported!


In [2]:
#Run the engine around the user_id 793

test_subject = '793'
k = 10

In [3]:
# Load our data set and compute the user similarity matrix
ml = MovieLens()
data = ml.load_movies_dataset()

#We use the [build_full_trainset()](https://surprise.readthedocs.io/en/stable/dataset.html#surprise.dataset.DatasetAutoFolds.build_full_trainset) from surprise to train on the whole dataset.

In [14]:
train_set = data.build_full_trainset()

#We use cosine similarity and user_based
sim_options = {'name': 'cosine',
               'user_based': True
               }


In [5]:
#Build the model using KNN basic with k =40 by defult. There are other algorithms such as 
#KNNWithMeans, KNNWithZScore, KNNBaseline
model = KNNBasic(sim_options=sim_options)

#Fit the model on the train_set
model.fit(train_set)

#Compute the similarity matrix
sims_matrix = model.compute_similarities()

Computing the cosine similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


In [6]:
# Get top N similar users to our test subject

test_user_inner_ID = train_set.to_inner_uid(test_subject)

similarity_row = sims_matrix[test_user_inner_ID]

In [7]:
test_user_inner_ID

610

In [8]:
similarity_row[:5]

array([0.9701425, 0.       , 0.       , 0.9486833, 1.       ])

In [9]:
similar_users = []
for innerID, score in enumerate(similarity_row):
    if (innerID != test_user_inner_ID):
        similar_users.append( (innerID, score) )

similar_users[:5]

[(0, 0.9701425001453319),
 (1, 0.0),
 (2, 0.0),
 (3, 0.9486832980505138),
 (4, 1.0)]

In [10]:
k_neighbors = []
for rating in similar_users:
    if rating[1] > 0.98:
        k_neighbors.append(rating)
        
k_neighbors[:5]

[(4, 1.0), (5, 1.0), (7, 1.0), (13, 1.0), (15, 0.9970544855015816)]

In [11]:
# Get the stuff they rated, and add up ratings for each item, weighted by user similarity
candidates = defaultdict(float)
for similar_users in k_neighbors:
    innerID = similar_users[0]
    user_similarity_score = similar_users[1]
    their_ratings = train_set.ur[innerID]
    for rating in their_ratings:
        candidates[rating[0]] += (rating[1] / 5.0) * user_similarity_score


In [12]:
# Build a dictionary of stuff the user has already seen
watched = {}
for itemID, rating in train_set.ur[test_user_inner_ID]:
    watched[itemID] = 1

In [13]:
# Get top-rated items from similar users:
print("We found movies you might like: ")
print("------------------------------")
pos = 0
for itemID, rating_sum in sorted(candidates.items(), key=itemgetter(1), reverse=True):
    if not itemID in watched:
        movieID = train_set.to_raw_iid(itemID)
        print(ml.get_movie_name(int(movieID)), rating_sum)
        pos += 1
        if (pos > 10):
            break


We found movies you might like: 
------------------------------
Shawshank Redemption, The (1994) 223.47883266615042
Forrest Gump (1994) 204.6865684359344
Silence of the Lambs, The (1991) 179.67734255097443
Matrix, The (1999) 173.27405052589975
Braveheart (1995) 150.42314467529476
Usual Suspects, The (1995) 144.59372102028402
Seven (a.k.a. Se7en) (1995) 142.74794212185282
Schindler's List (1993) 142.28924856846652
Star Wars: Episode IV - A New Hope (1977) 139.7184312328547
Terminator 2: Judgment Day (1991) 134.60949475351202
Jurassic Park (1993) 130.50161225904904
